# 11 · The Anatomy of Chaos — Eigenvector Analysis

**Observational record associated with the book**  
*Discovering Chaos in Prime Numbers — Computational Investigations through the Euler Mirror*  
© Alvaro Costa, 2025

This notebook is part of a canonical sequence of computational records.  
It introduces **no new hypotheses, conjectures, or interpretative models**.

Its purpose is exclusively to **record** the behaviour of arithmetic structures under an explicit,  
deterministic, and reproducible observational regime.

The complete conceptual discussion is presented in the book.  
This notebook documents only the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, adapted redistribution, or independent commercial use are not permitted.


---

## 1. Beyond the Eigenvalues

In the previous chapters, the analysis focused on the **eigenvalues** $ \lambda_i $ of the operator $ M $.  
They allowed us to characterise **where** spectral excitations are located and, through spacing statistics, to distinguish between  
uncorrelated regimes (Poisson) and correlated regimes (GOE).

In this notebook, the focus shifts to the **eigenvectors** $ v_i $.

While the eigenvalues describe the global spectral structure, the eigenvectors reveal **how this structure manifests internally**.  
They encode the “vibrational” patterns of the operator and allow us to assess whether the observed regime is merely statistically  
compatible or structurally **ergodic**.

---

## 2. Tool I — Participation Ratio ($PR$)

The *Participation Ratio* ($PR$) quantifies the degree of **delocalisation** of an eigenvector.

For a normalised eigenvector $ v = (v_1,\dots,v_N) $, it is defined as:

$
\mathrm{PR} = \frac{1}{\sum_{i=1}^{N} |v_i|^4}.
$

Interpretation:

* **Localised eigenvector**: concentrated on a few components $\to \text{small}\, \mathrm{PR}$ (of order unity).
* **Delocalised eigenvector**: spread across the full space $\to \mathrm{PR}$ proportional to $ N $.

In Random Matrix Theory, eigenvectors in the **GOE** class are ergodic.  
In this case, the normalised ratio

$
\frac{\mathrm{PR}}{N}
$

concentrates around the universal value:

$
\frac{1}{3}.
$

This value constitutes a **structural signature** of quantum chaos, independent of the detailed form of the operator.

---

## 3. Tool II — Statistics of the Components

A second, independent prediction of Random Matrix Theory concerns the **individual components** of the eigenvectors.

For matrices in the GOE class, the components of a typical *bulk* eigenvector behave as random variables drawn from a  
**real Gaussian distribution**, with zero mean.

In this notebook, this prediction is tested as follows:

1. An eigenvector from the centre of the spectrum is selected.
2. A histogram of its components is constructed.
3. The histogram is compared with a theoretical Gaussian curve.
4. The **Kolmogorov–Smirnov (KS) test** is applied to quantify compatibility.

A high *p-value* indicates that there is no statistical evidence to reject the Gaussian hypothesis.

---

## 4. The Eigenvector Laboratory

The following code cells implement these two analyses in a comparative manner.

Four plots are produced:

* **Linear scale**:

  * histogram of the components of an eigenvector;
  * distribution of the values of $ \mathrm{PR}/N $.

* **Logarithmic scale**:

  * histogram of the components of an eigenvector;
  * distribution of the values of $ \mathrm{PR}/N $.

The objective is to verify whether the distinction observed in the eigenvalues (Poisson versus GOE) also manifests itself in the  
**internal geometry** of the eigenvectors.

The conceptual interpretation of these results is presented in the text of Chapter 11.


In [1]:
# Requirements: pandas, matplotlib, numpy, ipywidgets, scipy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time
from scipy.stats import kstest, norm

# --- Data and Matrix Generation Functions (from the previous chapter) ---
def generate_pi_data(n: int) -> np.ndarray:
    """Generates an array with all primes up to n using an optimised sieve."""
    if n < 2:
        return np.array([], dtype=np.int64)
    size = (n - 1) // 2
    sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3
            start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]
    odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))


def get_delta_pi_for_points(x_points, primes):
    """Computes Δπ(x) for an array of x values using a precomputed prime list."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2


def generate_cos_matrix(fx_values, x_values):
    """Generates the matrix M from the vectors F(x) and x."""
    fx = fx_values.astype(np.float64)
    x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12
    logx = np.log(x)
    C = np.cos(np.outer(fx, logx))
    M = C + C.T
    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev
    return 0.5 * (M + M.T)


# Eigenvector analysis function
def calculate_participation_ratio(eigenvectors):
    """Computes the Participation Ratio for a matrix of eigenvectors."""
    # Eigenvectors returned by eigh are normalised (sum of squares equals 1)
    # Formula: PR = 1 / sum(|v_i|^4)
    return 1 / np.sum(eigenvectors**4, axis=0)


# --- Main Interactive Function ---
def eigenvector_analysis_lab(N=2048, log_X0=7):
    
    X0 = int(10**log_X0)
    span = 2.4  # Fixed span for the logarithmic scale
    
    # --- Data preparation ---
    max_x_log = int(np.ceil(X0 * np.exp(span/2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)
    pi_x_full = generate_pi_data(max_x_needed)

    # --- Linear scale analysis ---
    print("\n--- Processing Linear Scale ---")
    x_linear = np.arange(X0, X0 + N)
    fx_linear = get_delta_pi_for_points(x_linear, pi_x_full)

    # Jitter to ensure numerical stability
    jitter = 1e-9 * (fx_linear.std() if fx_linear.std() > 0 else 1)
    fx_linear_jittered = fx_linear.astype(np.float64) + np.random.uniform(
        -jitter, jitter, size=fx_linear.shape
    )

    M_linear = generate_cos_matrix(fx_linear_jittered, x_linear)
    lam_linear, v_linear = np.linalg.eigh(M_linear)
    
    # Eigenvector analysis
    vec_linear = v_linear[:, N // 2]  # Eigenvector from the spectral centre
    pr_linear = calculate_participation_ratio(v_linear)
    ks_linear = kstest(vec_linear * np.sqrt(N), 'norm')  # Components scaled by sqrt(N)

    # --- Logarithmic scale analysis ---
    print("\n--- Processing Logarithmic Scale ---")
    x_log = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
    fx_log = get_delta_pi_for_points(x_log, pi_x_full)

    M_log = generate_cos_matrix(fx_log, x_log)
    lam_log, v_log = np.linalg.eigh(M_log)

    # Eigenvector analysis
    vec_log = v_log[:, N // 2]  # Eigenvector from the spectral centre
    pr_log = calculate_participation_ratio(v_log)
    ks_log = kstest(vec_log * np.sqrt(N), 'norm')

    # --- Comparative plots (2x2 layout) ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(
        f"Eigenvector Analysis for N={N}, X₀≈{X0:g}",
        fontsize=18,
        weight='bold'
    )

    # --- Top row: Linear scale (Poisson) ---
    # a) Eigenvector components
    ax = axes[0, 0]
    ax.hist(vec_linear * np.sqrt(N), bins=50, density=True, alpha=0.7, label='Components')
    ax.set_title('a) Components of $v_i$ (Linear)', fontsize=14)
    ax.set_xlabel(r'Components $v_i \sqrt{N}$')
    ax.set_ylabel('Density')
    ax.text(
        0.05, 0.95,
        f'KS p-value: {ks_linear.pvalue:.2f}',
        transform=ax.transAxes,
        ha='left', va='top',
        color='red'
    )

    # b) Participation Ratio
    ax = axes[0, 1]
    ax.hist(pr_linear / N, bins=50, density=True, alpha=0.7, label='PR/N')
    mean_pr_linear = np.mean(pr_linear / N)
    ax.axvline(mean_pr_linear, color='red', ls='--', label=f'Mean = {mean_pr_linear:.3f}')
    ax.set_title('b) Participation Ratio (Linear)', fontsize=14)
    ax.set_xlabel('PR / N')
    ax.legend()

    # --- Bottom row: Logarithmic scale (GOE) ---
    # c) Eigenvector components
    ax = axes[1, 0]
    x_grid = np.linspace(-4, 4, 200)
    ax.hist(vec_log * np.sqrt(N), bins=50, density=True, alpha=0.7, label='Components')
    ax.plot(x_grid, norm.pdf(x_grid), 'r--', lw=2, label='Gaussian Theory')
    ax.set_title('c) Components of $v_i$ (Logarithmic)', fontsize=14)
    ax.set_xlabel(r'Components $v_i \sqrt{N}$')
    ax.set_ylabel('Density')
    ax.text(
        0.05, 0.95,
        f'KS p-value: {ks_log.pvalue:.2f}',
        transform=ax.transAxes,
        ha='left', va='top',
        color='green'
    )
    ax.legend()

    # d) Participation Ratio
    ax = axes[1, 1]
    ax.hist(pr_log / N, bins=50, density=True, alpha=0.7, label='PR/N')
    mean_pr_log = np.mean(pr_log / N)
    ax.axvline(mean_pr_log, color='red', ls='--', label=f'Mean = {mean_pr_log:.3f}')
    ax.axvline(1/3, color='green', ls=':', lw=3, label='GOE Theory ≈ 0.333')
    ax.set_title('d) Participation Ratio (Logarithmic)', fontsize=14)
    ax.set_xlabel('PR / N')
    ax.legend()

    fig.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


# --- Interactive widget ---
interact(
    eigenvector_analysis_lab,
    N=widgets.Dropdown(options=[512, 1024, 2048], value=1024, description='N:'),
    log_X0=widgets.IntSlider(
        min=5, max=8, step=1, value=7,
        description='X₀ = 10^',
        continuous_update=False
    )
);


interactive(children=(Dropdown(description='N:', index=1, options=(512, 1024, 2048), value=1024), IntSlider(va…

---

## 5. Analysis of the Results: Genuine Signal and Artefact

At first sight, the results obtained in the eigenvector laboratory appear paradoxical. This paradox, however, does not indicate an inconsistency  
in the method. On the contrary, it exposes with particular clarity the decisive importance of the **observational lens** employed throughout  
this work.

### Observation under the Logarithmic Scale

*(The genuine signal of chaos)*

* **Gaussian components (Panel c)**  
  The histogram of the components of an eigenvector extracted from the *bulk* of the spectrum fits the theoretical Gaussian curve consistently.  
  The Kolmogorov–Smirnov test yields a typical *p*-value of order `0.2`, well above the usual rejection threshold (`0.05`).  
  This indicates that **there is no statistical evidence to reject the Gaussian hypothesis**, in full agreement with the predictions of Random  
  Matrix Theory for GOE-type systems.

* **Participation Ratio close to 1/3 (Panel d)**  
  The distribution of the normalised values $\mathrm{PR}/N$ is strongly concentrated around the theoretical value `1/3`, with empirical means close to  
  this reference (for example, `≈ 0.32`).  
  This indicates that the eigenvectors are **delocalised and ergodic**, occupying the vector space in a homogeneous manner — a robust signature  
  of quantum chaos.

Together, these two independent results confirm that, under the logarithmic scale, the operator $ M $ exhibits not only spectral statistics  
compatible with the GOE, but also the **internal eigenvector structure** expected of genuinely chaotic systems.

---

### Observation under the Linear Scale

*(The “ghost in the machine”)*

* **“Perfectly” Gaussian components (Panel a)**  
  In an apparently counterintuitive manner, the KS test applied to the eigenvector components in the linear scale often produces even higher  
  *p*-values (for example, `≈ 0.7`).  
  Taken in isolation, this could suggest an even stronger form of chaos.

* **High Participation Ratio (Panel b)**  
  The mean value of PR/N also approaches `1/3`, a value typical of fully delocalised vectors.

Taken together, these observations give rise to an apparent paradox.

---

### Resolving the paradox: the role of *jitter*

In the linear scale, the matrix $ M $ constructed from $ \Delta_\pi(x) $ is structurally poor and highly degenerate.  
To make the numerical problem tractable, it becomes necessary to introduce a minimal random perturbation (*jitter*).

The effect of this perturbation is decisive:

> the eigenvectors come to reflect predominantly the statistical properties of the injected noise, rather than the underlying arithmetic  
> structure of the primes.

A generic random vector possesses, by construction:

* approximately Gaussian components;
* a Participation Ratio close to `1/3`.

Thus, in the linear regime, what is being measured is **an artefact of the regularisation procedure**, not a genuine physical or arithmetic signal.

The contrast with the logarithmic scale is crucial. In that regime, the matrix $ M $ is already rich and complex by construction, requiring no  
*jitter* whatsoever. Yet it displays the same signatures. This demonstrates that the GOE behaviour observed there is **intrinsic**, not  
artificially induced.

---

## 6. Future Perspectives: A Cryptographic Primitive?

The spectral and eigenvector properties observed in this work suggest applications that extend beyond purely theoretical analysis, including  
possible connections to cryptography.

Modern cryptography is founded on mathematical problems that are computationally easy in one direction but extremely difficult to invert — such  
as the factorisation of large integers.

The results of this chapter point towards the possibility of a **quantum-chaos-inspired cryptographic primitive**:

* **Central idea**  
  The system parameters — the initial point $ X_0 $, the dimension $ N $, and the functional $ \Delta_\pi(x) $ — could act as a  
  **private key**.
  From these parameters, it is computationally straightforward to generate specific eigenvectors of the matrix $ M $.

* **Potential security**  
  These eigenvectors are mathematical objects of extremely high complexity, with pseudo-random structure and extreme sensitivity to the  
  initial parameters.
  The inverse problem — reconstructing $ X_0 $ or the underlying arithmetic structure from a single eigenvector — appears, at first sight,  
  computationally intractable.

* **“Quantum” character**  
  The “quantum” nature of this proposal does not lie in the use of quantum hardware, but in the fact that the **statistics involved coincide  
  with those observed in chaotic quantum systems**, as described by the Montgomery–Odlyzko law.

These possibilities are not pursued further in the present work, but they point towards a promising direction: the direct instrumentalisation  
of the mathematics of quantum chaos, emerging from number theory, in concrete technological applications.
